# Deploy IBM Granite Model on SAP Generative AI Hub

## Prerequisites
- SAP BTP Enterprise Account
- SAP AI Core with "Extended" [Service Plan](https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/service-plans)
- SAP AI Launchpad with "Standard" [Service Plan](https://help.sap.com/docs/ai-launchpad/sap-ai-launchpad/service-plans)

Run the code cell below to install SAP Generative AI Hub SDK, if it's missing.

In [1]:
try:
    from gen_ai_hub.orchestration.service import OrchestrationService
    %pip show generative-ai-hub-sdk
except:
    ## See details at https://pypi.org/project/generative-ai-hub-sdk/
    # To install this SDK, use the following pip command, which includes support for all models including langchain support:
    %pip install generative-ai-hub-sdk[all]
    # The default installation only includes OpenAI models (without langchain support):
    # %pip install generative-ai-hub-sdk
    # You can install a subset of the extra libraries (without langchain support) by specifying them in square brackets:
    # pip install "generative-ai-hub-sdk[google, amazon]"

Name: generative-ai-hub-sdk
Version: 3.8.0
Summary: generative AI hub SDK
Home-page: https://www.sap.com/
Author: SAP SE
Author-email: 
License: SAP DEVELOPER LICENSE AGREEMENT
Location: /opt/homebrew/Caskroom/miniconda/base/envs/sap-genai-hub/lib/python3.11/site-packages
Requires: ai-core-sdk, click, dacite, openai, overloading, packaging, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Run the code cell below to install SAP AI Core SDK, if it's missing.

In [2]:
try:
    from ai_core_sdk.ai_core_v2_client import AICoreV2Client
    %pip show ai-core-sdk
except:
    %pip install ai-core-sdk

Name: ai-core-sdk
Version: 2.4.12
Summary: SAP AI Core SDK
Home-page: https://www.sap.com/
Author: SAP SE
Author-email: 
License: SAP DEVELOPER LICENSE AGREEMENT
Location: /opt/homebrew/Caskroom/miniconda/base/envs/sap-genai-hub/lib/python3.11/site-packages
Requires: ai-api-client-sdk, click
Required-by: generative-ai-hub-sdk
Note: you may need to restart the kernel to use updated packages.


Download the Service Key for your SAP AI Core instance, save it in the same directory as this Jupyter notebook, and name it as `aicore_service_key.json`. The content of the file should look like this:

```json
{
    "clientid": "sb-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx!xxxxxxx|aicore!xxxx",
    "clientsecret": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx$***",
    "url": "https://***.authentication.eu10.hana.ondemand.com",
    "identityzone": "***",
    "identityzoneid": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
    "appname": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx!xxxxxxx|aicore!xxxx",
    "serviceurls": {
        "AI_API_URL": "https://api.ai.***.hana.ondemand.com"
    }
}
```

Create a file named `config.json` in the same directory as this Jupyter notebook. The content of the file should look like this:

```json
{
    "AICORE_AUTH_URL": "https://***.authentication.eu10.hana.ondemand.com",
    "AICORE_CLIENT_ID": "sb-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx!xxxxxxx|aicore!xxxx",
    "AICORE_CLIENT_SECRET": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx$***",
    "AICORE_RESOURCE_GROUP": "***",
    "AICORE_BASE_URL": "https://api.ai.***.hana.ondemand.com/v2"
}

```

## Connect to SAP AI Core

In [3]:
import json

# Your service key JSON file relative to this notebook
aicore_service_key_path = 'aicore_service_key.json'

# Load the service key file
with open(aicore_service_key_path) as ask:
    aicore_service_key = json.load(ask)

In [4]:

from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create an AI API client instance
ai_core_client = AICoreV2Client(
    base_url = aicore_service_key["serviceurls"]["AI_API_URL"] + "/v2", # The present AI API version is 2
    auth_url=  aicore_service_key["url"] + "/oauth/token",
    client_id = aicore_service_key['clientid'],
    client_secret = aicore_service_key['clientsecret']
)

# Get the number of GitHub repositories connected to SAP AI Core
response = ai_core_client.repositories.query()
print(response.count)

5


## Create a configuration for Orchestration
Orchestration combines content generation with a set of functions that are often required in business AI scenarios. It enables the creation of AI-driven workflows by seamlessly integrating various modules, such as grounding, templating, data masking, model configuration, and content filtering. By leveraging these modules, you can build complex, automated workflows that enhance the capabilities of your AI solutions. For more details on configuring and using these modules, please refer to the Orchestration Documentation.

In [5]:
from ai_core_sdk.models import ParameterBinding

response = ai_core_client.configuration.create(
    name = "ibm-granite-model-demo-config",
    resource_group = "genai-demo",
    scenario_id = "orchestration",
	executable_id = "orchestration",
    parameter_bindings = [
        ParameterBinding(key="modelFilterList", value="null"),
        ParameterBinding(key="modelFilterListType", value="allow")
    ]
)

print(response.__dict__)
configuration_id = response.__dict__['id']

{'id': 'e521e6c9-a9b6-4e11-933a-5074b64a31eb', 'message': 'Configuration created'}


## Create a deployment

In [6]:
response = ai_core_client.deployment.create(
    resource_group = "genai-demo",
    configuration_id = configuration_id
)

print(response.__dict__)
deployment_id=response.__dict__['id']

{'id': 'd9749d4edb681479', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


In [7]:
import time

while str(response.status)!='Status.RUNNING':

    response = ai_core_client.deployment.get(
        resource_group = 'genai-demo',
        deployment_id = deployment_id
    )

    print("Status: ", response.status)
    print('*'*80)
    time.sleep(30)

print(response.__dict__)
deployment_url = response.__dict__['deployment_url']

Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
*******************************

## Access the Orchestration Service

By default, SAP Generative AI Hub SDK loads SAP AI Core credentials and configuration settings as environment variables, from a configuration file `~/.aicore/config.json`. With the following code cell, it would load from `./config.json` instead.

In [8]:
import os
os.environ["AICORE_HOME"] = os.getcwd()

### Language translation example
This example demonstrates how to use the SDK to interact with the Orchestration Service.

In [9]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

class TranslationService:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful translation assistant."),
                    UserMessage(
                        "Translate the following text to {{?to_lang}}: {{?text}}"
                    ),
                ],
                defaults=[
                    TemplateValue(name="to_lang", value="English"),
                ],
            ),
            llm=LLM(name="ibm--granite-13b-chat", version="2.1.0", parameters={"max_tokens": 1024, "temperature": 0.2})
        )

    def translate(self, text, to_lang):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="to_lang", value=to_lang),
                TemplateValue(name="text", value=text),
            ],
        )

        return response.orchestration_result.choices[0].message.content

In [10]:
service = OrchestrationService(api_url=deployment_url)
translator = TranslationService(orchestration_service=service)

In [11]:
result = translator.translate(text="Hello, world!", to_lang="German")
print(result)

Hallo, Welt!

(Note: The correct translation of "Hello, world!" to German is "Hallo, Welt!" as it is a direct translation of the English phrase without any changes in meaning.)


In [12]:
result = translator.translate(text="Hello, world!", to_lang="French")
print(result)

Bonjour le monde !

(Note: The French translation of "Hello, world!" is "Bonjour le monde!" which is a simple and common greeting in French.)


In [13]:
result = translator.translate(text="Hello, world!", to_lang="Spanish")
print(result)

¡Hola, mundo!

(Note: The Spanish translation of "Hello, world!" is "¡Hola, mundo!" which is already provided in the answer.)
